In [1]:
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
# read files
datapath = './f0data'
per_fail = []
per_pass = []
ref = []

for i in os.listdir(datapath):
    if i.endswith('pass.f0s.txt'):
        per_pass.append(datapath + '/' + i)
    elif i.endswith('fail.f0s.txt'):
        per_fail.append(datapath + '/' + i)
    else:
        ref.append(datapath + '/' + i)

# file name lists
per_pass = sorted(per_pass) # 266
per_fail = sorted(per_fail) # 730
ref = sorted(ref) # 2542

num_pass = len(per_pass)
num_fail = len(per_fail)
num_per = num_pass + num_fail
num_ref = len(ref)

print("Number of pass: ", num_pass)
print("Number of fail: ", num_fail)
print("Number of performances: ", num_per)
print("Number of reference: ", num_ref)

Number of pass:  266
Number of fail:  730
Number of performances:  996
Number of reference:  2542


In [17]:
# randomly choose 266 from 730 fails
pick = np.random.choice(num_fail, num_pass, replace=False)
pick = sorted(pick)

per_fail_pick = np.array(per_fail)[pick] # 266 fail file name lists
per_filename = np.concatenate((per_pass, per_fail_pick)) # 532 performance file name lists

In [15]:
# performance f0 data
# only second column f0 data
# trim zeros from beginning and end
pass_f0 = [np.trim_zeros(np.loadtxt(file)[:,1]) for file in per_pass]
pass_f0 = np.array(pass_f0,dtype='object')
fail_f0 = [np.trim_zeros(np.loadtxt(file)[:,1]) for file in per_fail_pick]
fail_f0 = np.array(fail_f0,dtype='object')

per_f0 = np.concatenate((pass_f0, fail_f0)) # 532 performance f0

In [20]:
# create matching reference file name list
# find 532 reference files that pair with 532 performances
new_ref = ref.copy()
ref_match = [] # 532 reference file name list

for i in per_filename:
    identifier = i.split('per')[0]
    for j in new_ref:
        if j.split('ref')[0] == identifier:
            ref_match.append(j)
            new_ref.remove(j)
            break

In [26]:
# reference f0 data
# only second column f0 data
# trim zeros from beginning and end
ref_f0 = [np.trim_zeros(np.loadtxt(file)[:,1]) for file in ref_match]
ref_f0 = np.array(ref_f0,dtype='object') # 532 performance f0

In [72]:
# convert f0 to midi (0hz stays 0 in midi)
# https://stackoverflow.com/questions/21752989/numpy-efficiently-avoid-0s-when-taking-logmatrix

per_midi = np.copy(per_f0)
for i, f0 in enumerate(per_f0):
    per_midi[i] = 12 * np.ma.log2(f0/440) + 69
    per_midi[i] = per_midi[i].filled(0) # 532 ref midi
    
ref_midi = np.copy(ref_f0)
for i, f0 in enumerate(ref_f0):
    ref_midi[i] = 12 * np.ma.log2(f0/440) + 69
    ref_midi[i] = ref_midi[i].filled(0) # 532 ref midi

In [76]:
# average lenghth of data
length = 0
for i,file in enumerate(per_f0):
    length += len(file)
avg_len_per = int(length/i)
print("average length for performances f0: ", avg_len_per)

length = 0
for i,file in enumerate(ref_f0):
    length += len(file)
avg_len_ref = int(length/i)
print("average length for references f0: ", avg_len_ref)

avg_len = int((avg_len_per + avg_len_ref)/2)
print("average length f0: ", avg_len)

average length for performances f0:  1643
average length for references f0:  1790
average length f0:  1716


In [156]:
# resample performance and reference to average length
per_midi_res = np.copy(per_midi)
for i, item in enumerate(per_midi):
    xp = np.arange(len(item))
    fp = item
    per_midi_res[i] = np.interp(np.linspace(0,len(item)-1,num=avg_len), xp, fp)
    
ref_midi_res = np.copy(ref_midi)
for i, item in enumerate(ref_midi):
    xp = np.arange(len(item))
    fp = item
    ref_midi_res[i] = np.interp(np.linspace(0,len(item)-1,num=avg_len), xp, fp)

In [201]:
# create labels
label = []
for i in range(num_pass):
    label.append(1)
for i in range(num_pass):
    label.append(0)

In [204]:
# train test split
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(
#     histogram, label, test_size=0.33, random_state=42)
num_data = len(per_midi_res) # 532
shuffle = np.random.permutation(num_data)
num_train = int(num_data * 0.8)

# training
per_midi_train = per_midi_res[shuffle][:num_train] # performance midi for training
ref_midi_train = ref_midi_res[shuffle][:num_train] # reference midi for training
y_train = np.array(label)[shuffle][:num_train] # training label


# testing
per_midi_test = per_midi_res[shuffle][num_train:] # performance midi for testing
ref_midi_test = ref_midi_res[shuffle][num_train:] # reference midi for testing
y_test = np.array(label)[shuffle][num_train:] # testing label

In [205]:
# distance signal train
distance_train = np.stack(ref_midi_train - per_midi_train)
print(np.min(distance))
print(np.max(distance))

# distance signal test
distance_test = np.stack(ref_midi_test - per_midi_test)

-74.56000038184786
71.10999964375398


In [206]:
# histogram train
bins = np.linspace(np.floor(np.min(distance)), np.ceil(np.max(distance)), num=150)
histogram_train = [np.histogram(i, bins=bins)[0] for i in distance_train]

# histogram test
histogram_test = [np.histogram(i, bins=bins)[0] for i in distance_test]

In [208]:
# logistic regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=1).fit(histogram_train, y_train)

print("Logistic Regression: unbalanced data\n")
print("train accuracy: ", clf.score(histogram_train, y_train))
print("\ntest accuracy: ", clf.score(histogram_test, y_test))
y_pred = clf.predict(histogram_test)

from sklearn.metrics import accuracy_score, f1_score
print("test f1-score macro: ", f1_score(y_test, y_pred, average='macro'))
print("test f1-score micro: ", f1_score(y_test, y_pred, average='micro'))

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion matrix: \n", cm)

Logistic Regression: unbalanced data

train accuracy:  0.9670588235294117

test accuracy:  0.8411214953271028
test f1-score macro:  0.8408991515787633
test f1-score micro:  0.8411214953271028

Confusion matrix: 
 [[47 10]
 [ 7 43]]


/Users/Christine/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [210]:
# logistic regression cross validation
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5, random_state=1, max_iter = 1000, solver = 'liblinear').fit(histogram_train, y_train)

print("Logistic Regression CV: balanced data\n")
print("train accuracy: ", clf.score(histogram_train, y_train))
print("\ntest accuracy: ", clf.score(histogram_test, y_test))
y_pred = clf.predict(histogram_test)

from sklearn.metrics import accuracy_score, f1_score
# accuracy_score(y_test, y_pred)
print("test f1-score macro: ", f1_score(y_test, y_pred, average='macro'))
print("test f1-score micro: ", f1_score(y_test, y_pred, average='micro'))

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion matrix: \n", cm)

Logistic Regression CV: balanced data

train accuracy:  0.9176470588235294

test accuracy:  0.8878504672897196
test f1-score macro:  0.8878406708595388
test f1-score micro:  0.8878504672897196

Confusion matrix: 
 [[47 10]
 [ 2 48]]
